# Import packages

# Auxiliary Functions

In [6]:
import numpy as np
from copy import deepcopy
from scipy.io import loadmat
import os
import time

In [7]:
def ldet(A):
    sign, value = np.linalg.slogdet(A)
    if sign > 0:
        return value
    else:
        return -np.inf
def ldet_objval(A,x):
    return ldet(np.dot(np.dot(A.T, np.diag(x.T[0])), A))

In [8]:
def init_binary(A,R,s,m,n):
    U, S, VH = np.linalg.svd(A, full_matrices=True)
    x = np.zeros((n,1))
    for j in range(n):
        for i in range(s):
            x[j] += (U[j,i]**2)
    x_save = deepcopy(x)
    x = np.zeros((n,1))
    for row in R:
        x[row] = 1
        x_save[row] = 0

    for i in range(s-m):
        max_indice = np.argmax(x_save)
        x[max_indice] = 1
        x_save[max_indice] = 0
    zlb = ldet_objval(A, x)
    xlb = x
    return xlb, zlb

def init_greedy(A,R,s,m,n):
    U, S, VH = np.linalg.svd(A, full_matrices=True)
    x = np.zeros((n,1))
    k = min(s,m)
    for j in range(n):
        for i in range(k):
            x[j] += (S[i] * U[j,i]**2)
    x_save = deepcopy(x)
    x = np.zeros((n,1))
    for row in R:
        x[row] = 1
        x_save[row] = 0

    for i in range(s-m):
        max_indice = np.argmax(x_save)
        x[max_indice] = 1
        x_save[max_indice] = 0

    zlb = ldet_objval(A, x)
    xlb = x
    return xlb, zlb

In [9]:
def LSFI(A,n,x_init,z_lb): # Local Search First Improvement
    x = deepcopy(x_init)
    flag = True
    while flag:
        flag = False
        for i in range(n):
            if x[i] > 0:
                x[i] = 0
                for j in range(n):
                    if j != i and x[j] == 0:
                        x[j] = 1
                        z_lb_new = ldet_objval(A, x)
                        if z_lb_new > z_lb:
                            z_lb = z_lb_new
                            flag = True
                            break
                        else:
                            x[j] = 0
                if flag:
                    break
                else:
                    x[i] = 1
    return x, z_lb

def LSFP(A,n,x_init,z_lb): # Local Search First Improvement Plus
    x = deepcopy(x_init)
    flag = True
    leave_x, enter_x = 0, 0
    while flag:
        flag = False
        for i in range(n):
            if x[i] > 0:
                x[i] = 0
                for j in range(n):
                    if j != i and x[j] == 0:
                        x[j] = 1
                        z_lb_new = ldet_objval(A, x)
                        if z_lb_new > z_lb:
                            leave_x, enter_x = i, j
                            z_lb = z_lb_new
                            flag = True
                        x[j] = 0
                if flag:
                    break
                else:
                    x[i] = 1
        if flag:
            # x[leave_x] = 0
            x[enter_x] = 1
    return x, z_lb

def LSBI(A,n,x_init,z_lb): # Local Search Best Improvement
    x = deepcopy(x_init)
    flag = True
    leave_x, enter_x = 0, 0
    while flag:
        flag = False
        for i in range(n):
            if x[i] > 0:
                x[i] = 0
                for j in range(n):
                    if j != i and x[j] == 0:
                        x[j] = 1
                        z_lb_new = ldet_objval(A, x)
                        if z_lb_new > z_lb:
                            leave_x, enter_x = i, j
                            z_lb = z_lb_new
                            flag = True
                        x[j] = 0
                x[i] = 1
        if flag:
            x[leave_x] = 0
            x[enter_x] = 1
    return x, z_lb


In [10]:
def run_local_search(A, R, n, m, s):
    x_init_bin, z_init_bin = init_binary(A, R, s, m, n)
    x_init_gre, z_init_gre = init_greedy(A, R, s, m, n)
    X = [x_init_bin, x_init_gre]
    Z = [z_init_bin, z_init_gre]

    X_init = [x_init_bin, x_init_gre]
    for x_init in X_init:
        x, z = LSFI(A, n, x_init, z_init_bin)
        X.append(x)
        Z.append(z)
        x, z = LSFP(A, n, x_init, z_init_bin)
        X.append(x)
        Z.append(z)
        x, z = LSBI(A, n, x_init, z_init_bin)
        X.append(x)
        Z.append(z)
    z_heur = np.max(Z)
    indsX = np.where(Z == z_heur)[0]
    x_heur = X[indsX[0]]

    sum_x = []
    max_x = []
    for x in X:
        sum_x.append(np.sum(x))
        max_x.append(np.max(x))
    return x_heur, z_heur, (X, Z, sum_x, max_x, indsX)    

In [14]:
instances = os.listdir('../instances')
for instance_name in instances:
        instance = loadmat(os.path.join('../instances', instance_name))
        A = instance["A"]
        n = A.shape[0]
        m = A.shape[1]
        s = int(n/2)
        R = instance['R']
        time_ini = time.time()
        x_ls, z_ls, info_ls = run_local_search(A, R, n, m, s)
        time_end = time.time()
        print(f"Finished LS {instance_name} - Time: {time_end - time_ini} - Result: {z_ls}")

Finished LS Instance_100_1.mat - Time: 7.943866729736328 - Result: -2.3847463607407398
Finished LS Instance_100_2.mat - Time: 6.092975378036499 - Result: -2.808564006102053
Finished LS Instance_100_3.mat - Time: 4.404726982116699 - Result: -2.6340154090524495
